In [ ]:
from sqlalchemy import create_engine
import geopandas as gp
import multiprocessing
import zipfile as zp
import pandas as pd
import numpy as np
import shutil
import os
from IPython.display import clear_output, display
import time
import concurrent.futures as cf
import psycopg2
from datetime import datetime
import geoalchemy2
#engine.dispose()


df = pd.read_csv(r'E:\combined.csv')
shp_df = df[df.Ext == 'shp']
shp_df.head()



# Function to read shape files into a temp directory
def shp_zip_read(full_path):
    os.makedirs(r"E:\New folder", exist_ok=True)
    zip_file_path = full_path.split('.zip\\')[0] + '.zip'
    shp_file_path = full_path.split('.zip\\')[1]
    print(shp_file_path)
    archive = zp.ZipFile(zip_file_path)
    if '/' not in shp_file_path:
        for file in archive.namelist():
            if '/' not in file:
                archive.extract(file, r"E:\New folder")
        shp_folder_path = ''
    else:
        shp_folder_path = '/'.join(shp_file_path.split('/')[:-1])+'/'
        for file in archive.namelist():
            if file.startswith(shp_folder_path):
                archive.extract(file, r"E:\New folder")
    gdf = gp.read_file("E:\\New folder\\{}".format(shp_folder_path))
    shutil.rmtree(r'E:\New folder', ignore_errors=True)
    return gdf


# Function to dump shape files into postgres db 
def shape_file_dump(file_name, file_path, engine):
    
    #connection = engine.connect()
    
    # Reading the shape file
    if '.zip' in file_path:
        gdf = shp_zip_read(file_path)
    else:
        gdf = gp.read_file(file_path)

    #Import shapefile to databse
    #print(file_name)
    gdf.to_postgis(name=file_name, con=engine, schema="postgis")
    
    #connection.close()
    

# Deriving a unique file name to create table in the database 
import hashlib
class FilePathShortener:
    def __init__(self):
        self.paths = {}
    
    def shorten_file_path(self, file_path):
        if file_path in self.paths:
            return self.paths[file_path]
        
        file_hash = hashlib.md5(file_path.encode()).hexdigest()
        shortened_path = file_hash[:8]
        
        self.paths[file_path] = shortened_path
        return shortened_path
    
    def decode_shortened_path(self, shortened_path):
        for path, short_path in self.paths.items():
            if short_path == shortened_path:
                return path
        
        return None

shortener = FilePathShortener()

 

"""file_path = '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india state_lcc.shp'
shortened_path = shortener.shorten_file_path(file_path)
print(shortened_path)"""

shortened_path = shp_df.File_path.apply(shortener.shorten_file_path) 
new_file_name = shp_df.File_Name+'_'+shortened_path
shp_df['unique_file_name']=new_file_name
        
   


# Getting the current date and time
#dt = datetime.now()
# getting the timestamp
#ts = datetime.timestamp(dt)
#print("Date and time is:", dt)
#current_time = dt.strftime("%d-%m-%Y %H:%M:%S")

    
if __name__ == '__main__':
    # Number of processes to run in parallel
    num_processes = multiprocessing.cpu_count()
        
    #DB details
    host = 'localhost'
    port = '5432'
    database = 'postgis_33'
    user = 'postgres'
    password = 'wadhwanikdss'

    conn = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(conn)
            
#     # Create a list to store the process instances
#     processes = []

#     for _ in range(num_processes):
#         p = multiprocessing.Process(target=shape_file_dump, args=(list(shp_df.File_Name), list(shp_df.File_path), engine))
#         p.start()
#         processes.append(p)

#     # Wait for all processes to finish
#     for p in processes:
#         p.join()
    
    j = 0
    
    
      
for i in range(8350,shp_df.shape[0]):
    #initialising empty data frame to store errors
    df = pd.DataFrame(columns=['iteration','unique_file_name', 'file_path', 'error', 'error_type']) 
        
    
    if '.zip' in shp_df.File_path.iloc[i]:  
        continue
    
    file_present = False
    #print('waiting for shared drive access ')
    while file_present == False:
        j=j+1
        file_check = r'\\192.168.1.151\ncfc_data3\CHAMAN\mango\East Champaran\analysis\merge_mango_echamparan.shp'
        if os.path.exists(file_check) or os.path.exists(file_check.replace(".151",".152")):
            print(f'Executing {i}st file')
            File_path = shp_df.File_path.iloc[i]
            
            if os.path.exists(File_path) == False:
                File_path = File_path.replace(".151",".152") 
            
            # Getting the current date and time
            dt = datetime.now()
           # getting the timestamp
            ts = datetime.timestamp(dt)
            current_time = dt.strftime("%d-%m-%Y %H:%M:%S")

            #Appending  unique_file_name column with current timestamp
            pg_file_name = shp_df.unique_file_name.iloc[i] +'_' +current_time
            shp_df.unique_file_name.iloc[i] = pg_file_name


            try:
                 shape_file_dump(shp_df.unique_file_name.iloc[i], File_path, engine)
        #print(shp_df.unique_file_name.iloc[i])

            except Exception as e:
                        error=e
                        print(error, File_path)
             #if 'Identifier' not in str(error):
            df.loc[len(df.index)] = [i,pg_file_name, File_path, error, type(error)]

            df.to_csv(f"C:/Users/user/Documents/Shape_file_DB_Upload_Error_tracker.csv" , mode='a', index=False, header=False)

                #if type(error) == fiona.errors.DriverError:
            file_present = True
            break 
            
        else:
            print('waiting for shared drive access ') 
            time.sleep(40)
    if j == 1000:
        time.sleep(3)
        clear_output(wait=True)
        display('Output cleared')
        print(i)
        j = 0     


print("Data dumped successfully into the PostGIS database.")
    
engine.dispose()

'Output cleared'

81658
Executing 81659st file


C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name
C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


Executing 81660st file
Executing 81661st file


C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name
C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


...

C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name
C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


Executing 82725st file
Executing 82726st file
Executing 82727st file


C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name
C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


Executing 82728st file
Executing 82729st file


C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name
C:\Users\user\AppData\Local\Temp\ipykernel_8520\1488168478.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name
